In [1]:
import torch
import torch.nn.functional as torch_functions
import matplotlib.pyplot as plt
from time import sleep
from IPython.display import clear_output


In [ ]:

class GameOfLife:
    def __init__(self, device):
        self.parameters = torch.zeros((1, 1, 3, 3), dtype=torch.float32, device=device)
        self.parameters[0, 0, :, :] = 1
        self.parameters[0, 0, 1, 1] = 9
        print(self.parameters)
   
    def __call__(self, state):
        next_state = torch_functions.pad(state, (1, 1, 1, 1), mode="circular")
        next_state = torch_functions.conv2d(next_state, self.parameters)
        next_state = ((next_state == 3) + (next_state == 11) + (next_state == 12)).to(torch.float32)
        # next_state[:, 0, :, :] = 1 - next_state[:, 1, :, :]
        return next_state



torch.random.seed()

state = torch.randint(0,2,[1,1,100,100]).to(torch.float32)
print(state.shape)

gol = GameOfLife(torch.get_default_device())

steps = 100

# states = np.zeros(state, )
for i in range(steps):
    state = gol(state)


# for i in range(steps):
    plt.figure(figsize=[10,10])
    plt.imshow(state[0,0])
    plt.show()
    sleep(0.1)
    clear_output(wait=True)


In [ ]:
class MP:
    def __init__(self, device):
        self.diff_parameters = torch.zeros((1, 3, 3, 3), dtype=torch.float32, device=device)
        self.flow_parameters = torch.zeros((1, 3, 3, 3), dtype=torch.float32, device=device)
        # self.parameters[0, 0, :, :] = [[-1, -1, -1],[-1, 0, -1],[-1, -1, -1],]
        # self.parameters[0, 0, 2, :] = [ torch.sqrt(2), -1, torch.sqrt(2)]
        # self.parameters[0, 0, 1, 1] = 9
        sq2 = torch.sqrt(torch.tensor(2.))
        self.diff_parameters = torch.tensor([
            [[[0, 0, 0], [0, sq2, 0], [-sq2, 0, 0]]],
            [[[0, 0, 0], [0, 1, 0], [0, -1, 0]]],
            [[[0, 0, 0], [0, sq2, 0], [0, 0, -sq2]]]
            ])
        self.flow_parameters = torch.tensor([
            [[[0, 0, 0], [0, 0, 0], [-sq2, 0, 0]]],
            [[[0, 0, 0], [0, 0, 0], [0, -1, 0]]],
            [[[0, 0, 0], [0, 0, 0], [0, 0, -sq2]]]
            ])
        # print(self.parameters)
   
    def __call__(self, state):
        # next_state = torch_functions.pad(state, (1, 1, 1, 1), mode="circular")

        heights, flows = state[0], state[1]
        diffs = torch_functions.conv2d(heights, self.diff_parameters)


        # either to all neighbors > 0 or to all neighbors equally according to their diff
        diffs = (diffs>0) * diffs #+ (diffs[:,0]<=0) * (diffs[:,1]<=0) * (diffs[:,2]>0) * diffs

        diffs = diffs / torch.sum(diffs, dim=0)

        print(diffs)

        flows = torch_functions.conv2d(flows, self.flow_parameters)

        print(heights[ :, 1:-1, 1:-1] .shape)
        print(flows.shape)
        heights[ :, 1:-1, 1:-1] = heights[ :, 1:-1, 1:-1] - 0.1 * flows

        state[0], state[1] = heights, flows
        # next_state[:, 0, :, :] = 1 - next_state[:, 1, :, :]
        return state
    








torch.random.seed()

state = torch.rand([2,1,100,100]).to(torch.float32)
print(state.shape)

mp = MP(torch.get_default_device())

steps = 100

# states = np.zeros(state, )
for i in range(steps):
    state = mp(state)


# for i in range(steps):
    plt.figure(figsize=[10,10])
    plt.imshow(state[0,0])
    plt.show()
    sleep(0.1)
    clear_output(wait=True)